In [1]:
from Datasets import BinChromaDataset
import numpy as np
from torch.utils.data import DataLoader, Subset
import sys
sys.path.insert(0, '..')
from transformer.models import TransformerFromModels, EncoderModel, DecoderModel
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
import torch
from tqdm import tqdm
import os

In [2]:
npz_path = '../data/augmented_and_padded_data.npz'
dataset = BinChromaDataset(npz_path)

split_idx = int( len(dataset)*0.8 )

train_set = Subset(dataset, range(0,split_idx))
test_set = Subset(dataset, range(split_idx, len(dataset)))

batch_size = 16
epochs = 1000

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)

In [3]:
src_vocab_size = 2**12
tgt_vocab_size = 2**12
d_model = 512
num_heads = 4
num_layers = 3
d_ff = 2048
max_seq_length = 129
dropout = 0.1

dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

encoderModel = EncoderModel(src_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)
decoderModel = DecoderModel(tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

encoderModel = encoderModel.to(dev)
decoderModel = decoderModel.to(dev)

transformer = TransformerFromModels(encoderModel, decoderModel)

transformer = transformer.to(dev)

In [ ]:
criterion = CrossEntropyLoss(ignore_index=0)
optimizer = Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

# keep best validation loss for saving
best_val_loss = np.inf
save_dir = '../saved_models/encoder_decoder_one_hot/'
encoder_path = save_dir + 'encoder_one_hot.pt'
decoder_path = save_dir + 'decoder_one_hot.pt'
os.makedirs(save_dir, exist_ok=True)

for epoch in range(epochs):
    train_loss = 0
    running_loss = 0
    samples_num = 0
    running_accuracy = 0
    accuracy = 0
    with tqdm(train_loader, unit='batch') as tepoch:
        tepoch.set_description(f"Epoch {epoch} | trn")
        for melodies, chords in tepoch:
            melodies = melodies.to(dev)
            chords = chords.to(dev)
            optimizer.zero_grad()
            output = transformer(melodies, chords[:, :-1])
            loss = criterion(output.contiguous().view(-1, tgt_vocab_size), chords[:, 1:].contiguous().view(-1))
            loss.backward()
            optimizer.step()
            # update loss
            samples_num += melodies.shape[0]
            running_loss += loss.item()
            train_loss = running_loss/samples_num
            # accuracy
            prediction = output.argmax(dim=2, keepdim=True).squeeze()
            running_accuracy += (prediction == chords[:, 1:]).sum().item()/prediction.shape[1]
            accuracy = running_accuracy/samples_num
            tepoch.set_postfix(loss=train_loss, accuracy=accuracy) # tepoch.set_postfix(loss=loss.item(), accuracy=100. * accuracy)
    # validation
    with torch.no_grad():
        val_loss = 0
        running_loss = 0
        samples_num = 0
        running_accuracy = 0
        accuracy = 0
        print('validation...')
        for melodies, chords in test_loader:
            melodies = melodies.to(dev)
            chords = chords.to(dev)
            output = transformer(melodies, chords[:, :-1])
            loss = criterion(output.contiguous().view(-1, tgt_vocab_size), chords[:, 1:].contiguous().view(-1))
            # update loss
            samples_num += melodies.shape[0]
            running_loss += loss.item()
            val_loss = running_loss/samples_num
            # accuracy
            prediction = output.argmax(dim=2, keepdim=True).squeeze()
            running_accuracy += (prediction == chords[:, 1:]).sum().item()/prediction.shape[1]
            accuracy = running_accuracy/samples_num
        if best_val_loss > val_loss:
            print('saving!')
            best_val_loss = val_loss
            torch.save(encoderModel.state_dict(), encoder_path)
            torch.save(decoderModel.state_dict(), decoder_path)
        print(f'validation: accuracy={accuracy}, loss={val_loss}')
    # print(f"Epoch: {epoch+1}, training loss: {train_loss} | validation loss {val_loss}")

Epoch 0 | trn: 100%|██████████████████████████████████| 267/267 [02:09<00:00,  2.06batch/s, accuracy=0.0456, loss=0.322]


validation...
saving!
validation: accuracy=0.07223997420262664, loss=0.2615051819727971


Epoch 1 | trn: 100%|██████████████████████████████████| 267/267 [02:24<00:00,  1.85batch/s, accuracy=0.0799, loss=0.242]


validation...
saving!
validation: accuracy=0.08576160881801126, loss=0.23122246739564647


Epoch 2 | trn: 100%|██████████████████████████████████| 267/267 [02:28<00:00,  1.80batch/s, accuracy=0.0914, loss=0.216]


...

Epoch 40 | trn: 100%|█████████████████████████████████| 267/267 [02:27<00:00,  1.81batch/s, accuracy=0.266, loss=0.0161]


validation...
validation: accuracy=0.09316369606003752, loss=0.3039305359963852


Epoch 41 | trn: 100%|█████████████████████████████████| 267/267 [02:27<00:00,  1.80batch/s, accuracy=0.267, loss=0.0156]


validation...
validation: accuracy=0.0933469160412758, loss=0.30462620115190686


Epoch 42 | trn:  69%|███████████████████████▎          | 183/267 [01:40<00:47,  1.77batch/s, accuracy=0.268, loss=0.015]

In [ ]:
# to load the model
encoderModel = EncoderModel(src_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)
decoderModel = DecoderModel(tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

encoderModel.load_state_dict(torch.load(encoder_dir))
decoderModel.load_state_dict(torch.load(decoder_dir))

transformer = TransformerFromModels(encoderModel, decoderModel)
# for inference, not retraining, we need to run
transformer.eval()